# Transfer weights from Learning to GradCAM
I'm not sure whether this work is necessary or not, but I did.

## What I did
I had only the weights for the left model, so I transfer the weights to the right model.

<table border="da">
  <thead>
    <tr>
      <th>For Learning</th>
      <th>For GradCAM</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><img src="./img/model-for-learning.png"></td>
      <td><img src="./img/model-for-GradCAM.png"></td>
    </tr>
  </tbody>
</table>

## Load Model

In [1]:
from main import generate_model, generate_GardCAM_model

### Left model (for learning)

In [2]:
weight_path = './weight/checkpoint_06-0.194.weight'

In [3]:
learning_model = generate_model(print_summary=False)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
learning_model.load_weights(weight_path)

### Right model (for GradCAM)

In [5]:
GradCAM_model = generate_GardCAM_model(print_summary=False)

```python
#Visualization.
from keras.utils.vis_utils import plot_model
plot_model(model, to_file="./img/model.png", show_shapes=True)
plot_model(GradCAM_model, to_file="./img/GradCAM-model.png", show_shapes=True)
```

***

## Transfer

In [6]:
import numpy as np
import pandas as pd

In [7]:
# model for learning
df_learning_model_layers = pd.DataFrame([(l.name,l.weights) for l in learning_model.layers], columns = ["layer_name","weight_num"])
df_learning_model_layers.weight_num = df_learning_model_layers.weight_num.apply(lambda x:len(x))
df_learning_model_layers[df_learning_model_layers.weight_num > 0]

,layer_name,weight_num
2,share_conv_1,2
5,share_conv_2,2
16,conv_3_1,2
17,conv_3_2,2
18,conv_4_1,2
19,conv_4_2,2
24,dense,2
25,dense_1,2


In [8]:
# model for GradCAM
df_GradCAM_model_layers = pd.DataFrame([(l.name,l.weights) for l in GradCAM_model.layers], columns = ["layer_name","weight_num"])
df_GradCAM_model_layers.weight_num = df_GradCAM_model_layers.weight_num.apply(lambda x:len(x))
df_GradCAM_model_layers[df_GradCAM_model_layers.weight_num > 0]

,layer_name,weight_num
2,x1_conv_1,2
3,x2_conv_1,2
6,x1_conv_2,2
7,x2_conv_2,2
18,conv_3_1,2
19,conv_3_2,2
20,conv_4_1,2
21,conv_4_2,2
26,dense_2,2
27,dense_3,2


***

We could understand what I did;
- `share_conv_1` → `x1_conv_1` and `x2_conv_1` (x1 and x2 are same weights)
- `share_conv_2` → `x1_conv_2` and `x2_conv_2` (x1 and x2 are same weights)
- `others` → `others`

In [9]:
# Make weights for GradCAM
GradCAM_weights = np.hstack([
    learning_model.get_weights()[:2],
    learning_model.get_weights()[:2],
    learning_model.get_weights()[2:4],
    learning_model.get_weights()[2:4],
    learning_model.get_weights()[4:]
])

In [10]:
# Transfer weights.
GradCAM_model.set_weights(GradCAM_weights)

***

## Save weights

```python
GradCAM_model.save_weights('./weight/GradCAM_model.hdf5')
learning_model.save_weights('./weight/learning_model.hdf5')
```